# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
# Number of particles considered
n_leptons = 1
n_photons = 2
n_jets = 1

def setup_observables(delphesprocessor):
    # ETMiss
    delphesprocessor.add_default_observables(
        n_leptons_max=n_leptons,
        n_photons_max=n_photons,
        n_jets_max=n_jets,
        include_met=True
    )
        
    # Correlations with MET
    for s1, i1 in [('l', 0), ('a', 0)]:
        delphesprocessor.add_observable(
            'deltaphi_{}{}_met'.format(s1, i1+1),
            'abs({}[{}].phi() - met.phi())'.format(s1, i1),
            required=True,
            default=0.
        )

    # Selected correlations between particles
    for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
        delphesprocessor.add_observable(
            'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
            required=True,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].eta - {}[{}].eta)'.format(s1, i1, s2, i2),
            required=True,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].phi() - {}[{}].phi())'.format(s1, i1, s2, i2),
            required=True,
            default=0.
        )
        delphesprocessor.add_observable(
            'signed_deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '{}[{}].charge *  abs({}[{}].phi() - {}[{}].phi())'.format(s1, i1, s1, i1, s2, i2),
            required=True
        )


In [5]:
def setup_cuts(delphesprocessor):
    delphesprocessor.add_cut(
        'n_ls > 0.1'
    )
    delphesprocessor.add_cut(
        'n_as > 0.1'
    )
    delphesprocessor.add_cut(
        'et_miss >= 10.'
    )

## Main loop

In [6]:
n_runs_per_benchmark = 18  # Number of run_cards

In [7]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5')
    
    # Load events
    for benchmark in dp.benchmark_names:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        dp.add_hepmc_sample(
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2'
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples()
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))
    
    # Clean up
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc'.format(run_str))
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events_delphes.root'.format(run_str))


11:14  
11:14  ------------------------------------------------------------
11:14  |                                                          |
11:14  |  MadMiner v2018.10.12                                    |
11:14  |                                                          |
11:14  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
11:14  |                                                          |
11:14  ------------------------------------------------------------
11:14  
11:14  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
11:14  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_02/tag_1_pythia8_events.hepmc.gz
11:14  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_03/tag_1_pythia8_events.hepmc.gz
11:14  Adding HepMC sample at /Use

11:17  Cut et_miss >= 10.: 9739 events pass, 261 events removed
11:17  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_05/tag_1_pythia8_events_delphes.root
11:17  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
11:17  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
11:17  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
11:17  Requiring existence of observable n_as: 10000 events pass, 0 events removed
11:17  Requiring existence of observable n_js: 10000 events pass, 0 events removed
11:17  Requiring existence of observable deltaphi_l1_met: 10000 events pass, 0 events removed
11:17  Requiring existence of observable deltaphi_a1_met: 10000 events pass, 0 events removed
11:17  Requiring existence of observable m_l1_a1: 10000 events pass, 0 events removed
11:17  Requiring existence of observable deltaeta_l1_a1: 10000 events pass, 

11:19  Requiring existence of observable deltaphi_l1_a1: 10000 events pass, 0 events removed
11:19  Requiring existence of observable signed_deltaphi_l1_a1: 6656 events pass, 3344 events removed
11:19  Cut n_ls > 0.1: 7784 events pass, 2216 events removed
11:19  Cut n_as > 0.1: 8670 events pass, 1330 events removed
11:19  Cut et_miss >= 10.: 9755 events pass, 245 events removed
11:19  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_09/tag_1_pythia8_events_delphes.root
11:19  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
11:19  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
11:19  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
11:19  Requiring existence of observable n_as: 10000 events pass, 0 events removed
11:19  Requiring existence of observable n_js: 10000 events pass, 0 events removed
11:19  Requiring existence of obs

11:20  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_17/tag_1_pythia8_events.hepmc.gz
11:20  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_18/tag_1_pythia8_events.hepmc.gz
11:21  Adding cut n_ls > 0.1
11:21  Adding cut n_as > 0.1
11:21  Adding cut et_miss >= 10.
11:21  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_13/tag_1_pythia8_events_delphes.root
11:21  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
11:21  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
11:21  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
11:21  Requ

11:21  Requiring existence of observable signed_deltaphi_l1_a1: 6918 events pass, 3082 events removed
11:21  Cut n_ls > 0.1: 8017 events pass, 1983 events removed
11:21  Cut n_as > 0.1: 8751 events pass, 1249 events removed
11:21  Cut et_miss >= 10.: 9736 events pass, 264 events removed
11:21  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_2.h5
11:21  Starting analysis of runs for card 3
11:21  
11:21  ------------------------------------------------------------
11:21  |                                                          |
11:21  |  MadMiner v2018.10.12                                    |
11:21  |                                                          |
11:21  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
11:21  |                                                          |
11:21  ----------------

...

17:27  Requiring existence of observable deltaeta_l1_a1: 10000 events pass, 0 events removed
17:27  Requiring existence of observable deltaphi_l1_a1: 10000 events pass, 0 events removed
17:27  Requiring existence of observable signed_deltaphi_l1_a1: 7523 events pass, 2477 events removed
17:27  Cut n_ls > 0.1: 8150 events pass, 1850 events removed
17:27  Cut n_as > 0.1: 9329 events pass, 671 events removed
17:27  Cut et_miss >= 10.: 9977 events pass, 23 events removed
17:27  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_89/tag_1_pythia8_events_delphes.root
17:27  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:27  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:27  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:27  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:27  Requiring existenc

17:30  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:30  Requiring existence of observable deltaphi_l1_met: 10000 events pass, 0 events removed
17:30  Requiring existence of observable deltaphi_a1_met: 10000 events pass, 0 events removed
17:30  Requiring existence of observable m_l1_a1: 10000 events pass, 0 events removed
17:30  Requiring existence of observable deltaeta_l1_a1: 10000 events pass, 0 events removed
17:30  Requiring existence of observable deltaphi_l1_a1: 10000 events pass, 0 events removed
17:30  Requiring existence of observable signed_deltaphi_l1_a1: 7561 events pass, 2439 events removed
17:30  Cut n_ls > 0.1: 8126 events pass, 1874 events removed
17:30  Cut n_as > 0.1: 9387 events pass, 613 events removed
17:30  Cut et_miss >= 10.: 9987 events pass, 13 events removed
17:30  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_93/tag_1_pythia8_events_delphes.root
17:30  

17:31  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_99/tag_1_pythia8_events.hepmc.gz
17:31  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_100/tag_1_pythia8_events.hepmc.gz
17:32  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_101/tag_1_pythia8_events.hepmc.gz
17:32  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_102/tag_1_pythia8_events.hepmc.gz
17:33  Adding cut n_ls > 0.1
17:33  Addin

17:33  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:33  Requiring existence of observable deltaphi_l1_met: 10000 events pass, 0 events removed
17:33  Requiring existence of observable deltaphi_a1_met: 10000 events pass, 0 events removed
17:33  Requiring existence of observable m_l1_a1: 10000 events pass, 0 events removed
17:33  Requiring existence of observable deltaeta_l1_a1: 10000 events pass, 0 events removed
17:33  Requiring existence of observable deltaphi_l1_a1: 10000 events pass, 0 events removed
17:33  Requiring existence of observable signed_deltaphi_l1_a1: 7517 events pass, 2483 events removed
17:33  Cut n_ls > 0.1: 8103 events pass, 1897 events removed
17:33  Cut n_as > 0.1: 9372 events pass, 628 events removed
17:33  Cut et_miss >= 10.: 9989 events pass, 11 events removed
17:33  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects

17:36  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:36  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:36  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:36  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:36  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:36  Requiring existence of observable deltaphi_l1_met: 10000 events pass, 0 events removed
17:36  Requiring existence of observable deltaphi_a1_met: 10000 events pass, 0 events removed
17:36  Requiring existence of observable m_l1_a1: 10000 events pass, 0 events removed
17:36  Requiring existence of observable deltaeta_l1_a1: 10000 events pass, 0 events removed
17:36  Requiring existence of observable deltaphi_l1_a1: 10000 events pass, 0 events removed
17:36  Requiring existence of observable signed_deltaphi_l1_a1: 7513 events pass, 2487 events removed
17:36  Cut n_ls 

## Combine samples

In [8]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

17:36  
17:36  ------------------------------------------------------------
17:36  |                                                          |
17:36  |  MadMiner v2018.10.12                                    |
17:36  |                                                          |
17:36  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
17:36  |                                                          |
17:36  ------------------------------------------------------------
17:36  
17:36  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
17:36  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
17:36  Loading samples 